Library

In [30]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datetime import datetime
from datetime import timedelta
from sklearn.model_selection import cross_val_score, GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
#naive bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix



In [31]:
df = pd.read_csv("comment_final.csv", encoding="utf-8", index_col=0)

Check NAN


In [32]:
print('Data shape:', df.shape)
missing_val_num = df.isnull().sum()
missing_percent = (missing_val_num / df.shape[0]) * 100
missing_info = {'missing_values': missing_val_num,
                'missing_percent': round(missing_percent,3)}
missing_df = pd.DataFrame(missing_info)
missing_df

Data shape: (85999, 2)


,missing_values,missing_percent
processed_text,4849,5.638
class,0,0.000


In [33]:
df.dropna(inplace=True)
print(df.shape)
print(df.isnull().sum())

(81150, 2)
processed_text    0
class             0
dtype: int64


Duplicate Data


In [34]:
df[df.duplicated()]

,processed_text,class
365,không_bàn nội đọc đóng_gói cạn vận_chuyển đườn...,negative
465,đóng gói sơ_sài sách hộp cactong chống sốc sác...,positive
470,không_bàn nội đọc đóng_gói cạn vận_chuyển đườn...,negative
474,sách đợt dọn nhã nam đợt lắm rùi cười trading ...,positive
604,sách hàng sách yên_tâm,positive
...,...,...
85993,đóng gói đẹp,positive
85994,sách đẹp,positive
85996,sách đẹp,positive
85997,lắm,positive


In [35]:
# Drop all duplicate rows
df.drop_duplicates(inplace=True)
print('Duplicate data:', df.duplicated().sum())
print(df.shape)
# Reset index
df.reset_index(drop=True, inplace=True)

Duplicate data: 0
(66886, 2)


In [38]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['processed_text'])
X

<66886x26463 sparse matrix of type '<class 'numpy.int64'>'
	with 704434 stored elements in Compressed Sparse Row format>

In [26]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
x_train_tfidf = tfidf_vectorizer.fit_transform(x_train)
x_test_tfidf = tfidf_vectorizer.transform(x_test)


In [28]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(x_train_tfidf, y_train)


MultinomialNB()

In [29]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(x_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8649955148011562


In [ ]:
df_new.head()

In [ ]:
y = df['class']

In [ ]:
print(X_int.shape, y.shape)

In [ ]:
# Choose 0.3, 0.2 as the test size
test_size_list = [.3, .2]
# Run 5 times for each test_size
for test_size in test_size_list:
  print('Test size: ', test_size)
  accuracy_train_list = []
  accuracy_test_list = []
  for i in range(5):
    print('Run: ', i)
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_int, y, test_size=test_size)
    # Create a LogisticReg (for faster training)
    clf = MultinomialNB()
    # Train the classifier
    clf.fit(X_train, y_train)
    # Make predictions
    y_pred = clf.predict(X_test)
    # Calculate the accuracy
    accuracy_train = accuracy_score(y_train, clf.predict(X_train))
    accuracy_test = accuracy_score(y_test, y_pred)
    accuracy_train_list.append(accuracy_train)
    accuracy_test_list.append(accuracy_test)
  print('Test size: ', test_size, ' Train accuracy: ', sum(accuracy_train_list) / len(accuracy_train_list), ' Test accuracy: ',
      sum(accuracy_test_list) / len(accuracy_test_list))
  print('--------------------------------------------------')

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_int, y, test_size=0.2, random_state=42)

In [ ]:
#naive bayes model
models = [MultinomialNB()]


In [ ]:
def get_model_name(model):
    model_name = model.__class__.__name__
    if model_name == 'MultinomialNB':
        return model_name 

In [ ]:
# Number of folds for cross validation
num_folds = 5
# A dataframe to store the results
results = pd.DataFrame(columns=['Model', 'Accuracy Mean', 'Accuracy 3 * STD', 'Time'])
# Integrate through models
for model in models:
    # Get model name
    model_name = get_model_name(model)  
    print('Model: ', model_name, 'running...')
    # Get the current time
    start_time = datetime.now()  
    # Perform cross validation
    cv_results = cross_val_score(model, X_train, y_train, cv=num_folds, scoring='accuracy', n_jobs=-1)  
    # Get the training time
    train_time = datetime.now() - start_time  
    # Get the mean and std of the results
    mean_cv_results = cv_results.mean()
    std_cv_results = cv_results.std()  
    # Store the results
    results = results.append({'Model': model_name,
                                'Accuracy Mean': mean_cv_results,
                                'Accuracy 3 * STD': 3 * std_cv_results,
                                'Time': train_time
                                }, ignore_index=True)

In [ ]:
# Sort the dataframe by Accuracy Mean in descending order
results = results.sort_values(by='Accuracy Mean', ascending=False).reset_index(drop=True)
results

Tuning


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the range of hyperparameters to search over
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 5.0],
    'fit_prior': [True, False]
}

# Create the MultinomialNB model
clf = MultinomialNB()

# Create a GridSearchCV object and fit it to the training data
search_clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=-1)
search_clf.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding accuracy score
print('Best params:', search_clf.best_params_)
print('Best accuracy:', search_clf.best_score_)

# Create the best model with optimal parameters
best_alpha = search_clf.best_params_['alpha']
best_fit_prior = search_clf.best_params_['fit_prior']
best_model = MultinomialNB(alpha=best_alpha, fit_prior=best_fit_prior)

# Train the best model on the training data
best_model.fit(X_train, y_train)

# Predict on the test data using the best model
y_pred = best_model.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Test accuracy:', accuracy)


In [ ]:
accuracy_train = accuracy_score(y_train, best_model.predict(X_train))
accuracy_test = accuracy_score(y_test, y_pred)
print('Train accuracy: ', accuracy_train, ' Test accuracy: ', accuracy_test)



In [ ]:
from sklearn.metrics import classification_report,ConfusionMatrixDisplay
print(classification_report(y_test,y_pred))
ConfusionMatrixDisplay.from_predictions(y_test,y_pred);


In [ ]:
from yellowbrick.classifier import ConfusionMatrix, ClassificationReport, ROCAUC, roc_auc
visualizer = ROCAUC(MultinomialNB(), classes=y.unique())

visualizer.fit(X_train, y_train)        # Fit the training data to the visualizer
visualizer.score(X_test, y_test)        # Evaluate the model on the test data
visualizer.show()